## Computing PCA using RDDs

##  PCA

The vectors that we want to analyze have length, or dimension, of 365, corresponding to the number of 
days in a year.

We will perform [Principle component analysis (PCA)](https://en.wikipedia.org/wiki/Principal_component_analysis)
on these vectors. There are two steps to this process:

1) Computing the covariance matrix: this is a  simple computation. However, it takes a long time to compute and it benefits from using an RDD because it involves all of the input vectors.

2) Computing the eigenvector decomposition. this is a more complex computation, but it takes a fraction of a second because the size to the covariance matrix is $365 \times 365$, which is quite small. We do it on the head node usin `linalg`

### Computing the covariance matrix
Suppose that the data vectors are the column vectors denoted $x$ then the covariance matrix is defined to be
$$
E(x x^T)-E(x)E(x)^T
$$

Where $x x^T$ is the **outer product** of $x$ with itself.

If the data that we have is $x_1,x_2,x_n$ then  we estimate the covariance matrix:
$$
\hat{E}(x x^T)-\hat{E}(x)\hat{E}(x)^T
$$

the estimates we use are:
$$
\hat{E}(x x^T) = \frac{1}{n} \sum_{i=1}^n x_i x_i^T,\;\;\;\;\;
\hat{E}(x) = \frac{1}{n} \sum_{i=1}^n x_i
$$

## Computing the covariance matrix where the `nan`s are
### The effect of  `nan`s in arithmetic operations
* We use an RDD of numpy arrays, instead of Dataframes.
* Why? Because unlike dataframes, `numpy.nanmean` treats `nan` entries correctly.

### Calculating the mean of a vector with nan's
* We often get vectors $x$ in which some, but not all, of the entries are `nan`. 
* We want to compute the mean of the elements of $x$. 
* If we use `np.mean` we will get the result `nan`. 
* A useful alternative is to use `np.nanmean` which removes the `nan` elements and takes the mean of the rest.

In [1]:
import numpy as np
a=np.array([1,np.nan,2,np.nan,3,4,5])
print('a=',a)
print('np.mean(a)=',np.mean(a))
print('np.mean(np.nan_to_num(a))=',np.mean(np.nan_to_num(a))) # =(1+0+2+0+3+4+5)/7
print('np.nanmean(a)=',np.nanmean(a)) # =(1+2+3+4+5)/5

a= [ 1. nan  2. nan  3.  4.  5.]
np.mean(a)= nan
np.mean(np.nan_to_num(a))= 2.142857142857143
np.nanmean(a)= 3.0


### The outer poduct of a vector with `nan`s with itself

In [2]:
np.outer(a,a)

array([[ 1., nan,  2., nan,  3.,  4.,  5.],
       [nan, nan, nan, nan, nan, nan, nan],
       [ 2., nan,  4., nan,  6.,  8., 10.],
       [nan, nan, nan, nan, nan, nan, nan],
       [ 3., nan,  6., nan,  9., 12., 15.],
       [ 4., nan,  8., nan, 12., 16., 20.],
       [ 5., nan, 10., nan, 15., 20., 25.]])

### When should you not use `np.nanmean` ?
Using `n.nanmean` is equivalent to assuming that choice of which elements to remove is independent of the values of the elements. 
* Example of bad case: suppose the larger elements have a higher probability of being `nan`. In that case `np.nanmean` will under-estimate the mean

### Computing the covariance  when there are `nan`s
The covariance is a mean of outer products.

We calculate two matrices:
* $S$ - the sum of the matrices, whereh `nan`->0
* $N$ - the number of not-`nan` element for each matrix location.

We then calculate the mean as $S/N$ (division is done element-wise)

## Computing the mean together with the covariance
To compute the covariance matrix we need to compute both $\hat{E}(x x^T)$ and $\hat{E}(x)$. Using a simple trick, we can compute both at the same time.

Here is the trick: lets denote a $d$ dimensional **column vector** by $\vec{x} = (x_1,x_2,\ldots,x_d)$ (note that the subscript here is the index of the coordinate, not the index of the example in the training set as used above). 

The augmented vector $\vec{x}'$ is defined to be the $d+1$ dimensional vector $\vec{x}' = (1,x_1,x_2,\ldots,x_d)$.

The outer product of $\vec{x}'$ with itself is equal to 

$$ \vec{x}' {\vec{x}'}^T
= \left[\begin{array}{c|ccc}
    1 &  &{\vec{x}}^T &\\
    \hline \\
    \vec{x} & &\vec{x} {\vec{x}}^T \\ \\
    \end{array}
    \right]
$$

Where the lower left matrix is the original outer product $\vec{x} {\vec{x}}^T$ and the first row and the first column are $\vec{x}^T$ and $\vec{x}$ respectively.

Now suppose that we take the average of the outer product of the augmented vector and convince yourself that:
$$
\hat{E}(\vec{x}' {\vec{x}'}^T) = \frac{1}{n} \sum_{i=1}^n {\vec{x}'}_i {\vec{x}'}_i^T
= \left[\begin{array}{c|ccc}
    1 &  &\hat{E}(\vec{x})^T &\\
    \hline \\
    \hat{E}(\vec{x}) & &\hat{E}(\vec{x} {\vec{x}}^T) \\ \\
    \end{array}
    \right]
$$

So indeed, we have produced the outer product average together with (two copies of) the average $\hat{E}(\vec{x})$

In [3]:
import os
import sys
os.environ["PYSPARK_PYTHON"] = sys.executable
os.environ["PYSPARK_DRIVER_PYTHON"] = sys.executable

from pyspark import SparkContext,SparkConf

def create_sc(pyFiles):
    sc_conf = SparkConf()
    sc_conf.setAppName("Weather_PCA")
    sc_conf.set('spark.executor.memory', '3g')
    sc_conf.set('spark.executor.cores', '1')
    sc_conf.set('spark.cores.max', '4')
    sc_conf.set('spark.default.parallelism','10')
    sc_conf.set('spark.logConf', True)
    print(sc_conf.getAll())

    sc = SparkContext(conf=sc_conf,pyFiles=pyFiles)

    return sc 

sc = create_sc(pyFiles=['lib/numpy_pack.py','lib/spark_PCA.py','lib/computeStatistics.py'])

dict_items([('spark.app.name', 'Weather_PCA'), ('spark.executor.memory', '3g'), ('spark.executor.cores', '1'), ('spark.cores.max', '4'), ('spark.default.parallelism', '10'), ('spark.logConf', 'True')])


In [4]:
from pyspark.sql import *
sqlContext = SQLContext(sc)

import numpy as np
from lib.computeStatistics import *

### Climate data

The data we will use here comes from [NOAA](https://www.ncdc.noaa.gov/). Specifically, it was downloaded from This [FTP site](ftp://ftp.ncdc.noaa.gov/pub/data/ghcn/daily/).

There is a large variety of measurements from all over the world, from 1870 will 2012.
in the directory `../../Data/Weather` you will find the following useful files:

* data-source.txt: the source of the data
* ghcnd-readme.txt: A description of the content and format of the data
* ghcnd-stations.txt: A table describing the Meteorological stations.



Most of the information presented above about the NOAA files is no longer valid. The file format and links have changed dramatically in recent years.

All the old files needed to reproduce the results were saved locally. If you want to explore further to see the updated NOAA weather data, please see [this](https://registry.opendata.aws/noaa-ghcn/). The data files are avaiable and updated daily at a [S3 bucket](https://noaa-ghcn-pds.s3.amazonaws.com/index.html).

### Data cleaning

* Most measurements exists only for a tiny fraction of the stations and years. We therefor restrict our use to the following measurements:
```python
['TMAX', 'SNOW', 'SNWD', 'TMIN', 'PRCP', 'TOBS']
```

* 8 We consider only measurement-years that have at most 50 `NaN` entries

* We consider only measurements in the continential USA

* We partition the stations into the states of the continental USA (plus a few stations from states in canada and mexico).

In [5]:
state='NY'
data_dir='../resource/asnlib/publicdata/Data'
parquet=state+'.parquet'

In [6]:
parquet_path = data_dir+'/'+parquet
!du -sh $parquet_path

77M	../resource/asnlib/publicdata/Data/NY.parquet


In [7]:
%%time
df=sqlContext.read.parquet(parquet_path)
print(df.count())
df.show(5)

168398
+-----------+-----------+----+--------------------+-----+
|    Station|Measurement|Year|              Values|state|
+-----------+-----------+----+--------------------+-----+
|USC00305635|   TMAX_s20|1969|[00 00 00 00 00 0...|   NY|
|USC00305635|   TMAX_s20|1970|[A0 CB 03 CC 26 C...|   NY|
|USC00305635|   TMAX_s20|1971|[47 BF F5 C1 24 C...|   NY|
|USC00305635|   TMAX_s20|1972|[0D 51 ED 50 CF 5...|   NY|
|USC00305635|   TMAX_s20|1973|[6A 4F 66 4F 62 4...|   NY|
+-----------+-----------+----+--------------------+-----+
only showing top 5 rows

CPU times: user 0 ns, sys: 4 ms, total: 4 ms
Wall time: 4.03 s


In [8]:
sqlContext.registerDataFrameAsTable(df,'table')

In [9]:
Query="""
SELECT Measurement,count(Measurement) as count 
FROM table 
GROUP BY Measurement
ORDER BY count
"""
counts=sqlContext.sql(Query)
counts.show()

+-----------+-----+
|Measurement|count|
+-----------+-----+
|       TOBS|10956|
|   TOBS_s20|10956|
|   TMAX_s20|13437|
|       TMAX|13437|
|       TMIN|13442|
|   TMIN_s20|13442|
|       SNWD|14617|
|   SNWD_s20|14617|
|   SNOW_s20|15629|
|       SNOW|15629|
|       PRCP|16118|
|   PRCP_s20|16118|
+-----------+-----+



In [10]:
# The details of computeStatistics are in 
# lib/computeStatistics.py
# lib/spark_PCA.py

In [11]:
%%time 
### This is the main cell, where all of the statistics are computed.
STAT=computeStatistics(sqlContext,df)

TMAX : shape of mdf is  13437
time for TMAX is 73.2228193283081
SNOW : shape of mdf is  15629
time for SNOW is 54.122116565704346
SNWD : shape of mdf is  14617
time for SNWD is 46.12119817733765
TMIN : shape of mdf is  13442
time for TMIN is 42.32114362716675
PRCP : shape of mdf is  16118
time for PRCP is 61.518940448760986
TOBS : shape of mdf is  10956
time for TOBS is 41.94804048538208
CPU times: user 7.54 s, sys: 9.72 s, total: 17.3 s
Wall time: 5min 19s


In [12]:
from time import time
t=time()

N=sc.defaultParallelism
print('Number of executors=',N)
print('took',time()-t,'seconds')

Number of executors= 10
took 0.0014851093292236328 seconds


In [13]:
print("   Name  \t                 Description             \t  Size")
print("-"*80)
print('\n'.join(["%10s\t%40s\t%s"%(s[0],s[1],str(s[2])) for s in STAT_Descriptions]))

   Name  	                 Description             	  Size
--------------------------------------------------------------------------------
SortedVals	                        Sample of values	vector whose length varies between measurements
     UnDef	      sample of number of undefs per row	vector whose length varies between measurements
      mean	                              mean value	()
       std	                                     std	()
    low100	                               bottom 1%	()
   high100	                                  top 1%	()
   low1000	                             bottom 0.1%	()
  high1000	                                top 0.1%	()
         E	                   Sum of values per day	(365,)
        NE	                 count of values per day	(365,)
      Mean	                                    E/NE	(365,)
         O	                   Sum of outer products	(365, 365)
        NO	               counts for outer products	(365, 365)
       Cov	                

In [14]:
## Dump STAT and STST_Descriptions into a pickle file.
from pickle import dump

output_dir='./Outputs'
os.makedirs(output_dir,exist_ok=True)
filename=output_dir+'/STAT_%s.pickle'%state

with open(filename, 'wb') as file:
    dump((STAT,STAT_Descriptions),file)

In [15]:
X=STAT['TMAX']['Var']
for key in STAT.keys():
    Y=STAT[key]['Var']
    print(key,sum(abs(X-Y)))

TMAX 0.0
SNOW 852107.7058667188
SNWD 4464167.852118857
TMIN 319734.53150046256
PRCP 1184305.1228400553
TOBS 277719.0089381143


### Summary
* We discussed how to compute the covariance matrix and the expectation matrix when there are `nan` entries.
* The details are all in `computeStatistics`, which is defined in python files you can find in the directory `lib`